In [2]:
from demoparser import DemoParser
import pandas as pd

parser = DemoParser(r"C:\Program Files (x86)\Steam\steamapps\common\Counter-Strike Global Offensive\csgo\replays\match730_003616389815731224751_1839561675_182.dem")
events = parser.parse_events("smokegrenade_detonate")
events = pd.DataFrame(events)

In [8]:
events = events.drop(['entityid', 'event_name'], axis=1)
events2 = events.drop(['player_steamid','tick','player_name','round'], axis=1)
events2 = events2.values.flatten()

In [ ]:
pd.set_option('display.max_rows', 166)
display(events)
pd.reset_option('display.max_rows')

In [9]:
a = parser.parse_players()
names = [entry['name'] for entry in a]
print(names)
country_map = {country: index for index, country in enumerate(names)}
country_map

['P1STON', 'Sehnsucht', 'Fmgeez', 'jadasek', 'OE', 'Vladiclavchik', 'EO g4skins', 'Vampirus', 'Wargon', 'Feen']


{'P1STON': 0,
 'Sehnsucht': 1,
 'Fmgeez': 2,
 'jadasek': 3,
 'OE': 4,
 'Vladiclavchik': 5,
 'EO g4skins': 6,
 'Vampirus': 7,
 'Wargon': 8,
 'Feen': 9}

In [22]:
events.to_excel('smokes222.xlsx')

In [290]:
df.iloc[78571]

X                           -734.983398
Y                          -1141.940552
Z                           11648.03125
health                                0
spotted                               0
equipment_value                    4200
round_start_eq_val                 3000
kills                               4.0
ammo                                NaN
flash_duration                      0.0
tick                              16337
steamid               76561197994427720
name                          Sehnsucht
Name: 78571, dtype: object

In [10]:
import numpy as np
from sklearn.preprocessing import LabelEncoder


ticks = events['tick'].values
wanted_props = ["X", "Y", "Z", "health", 'spotted','equipment_value','round_start_eq_val', 'kills','ammo','flash_duration']
df = parser.parse_ticks(wanted_props)
df = pd.DataFrame(df)

smokes_new = []
indexing = 0
for i in ticks:
    index_of_tick = df.index[df['tick'] == i][0]
    
    index_of_tick_start = index_of_tick - 2600
    index_of_tick_end = index_of_tick + 7199
    new_df = df.loc[index_of_tick_start:index_of_tick_end] 

    print(index_of_tick_start, index_of_tick_end)
    new_df = new_df.drop(['steamid','tick'], axis=1)
    combined_row = new_df.values.flatten()
    combined_row = np.append(combined_row, events2[indexing:indexing+3])
    smokes_new.append(combined_row)

    indexing += 3


54392 64191
60582 70381
63052 72851
76832 86631
81322 91121
83572 93371
100972 110771
101632 111431
109092 118891
125882 135681
139552 149351
163392 173191
163472 173271
163872 173671
182992 192791
183422 193221
194952 204751
195892 205691
197862 207661
206982 216781
207312 217111
226232 236031
226392 236191
258182 267981
258382 268181
258812 268611
263462 273261
264462 274261
279432 289231
280502 290301
280992 290791
296772 306571
300212 310011
306412 316211
317912 327711
325852 335651
328572 338371
342972 352771
344112 353911
344212 354011
344602 354401
355682 365481
363712 373511
366782 376581
383552 393351
384342 394141
421112 430911
421742 431541
424682 434481
438432 448231
449602 459401
450612 460411
465922 475721
466282 476081
486562 496361
489292 499091
508162 517961
534262 544061
534472 544271
535042 544841
536432 546231
556622 566421
556912 566711
558642 568441
583712 593511
584512 594311
613322 623121
613422 623221
614532 624331
617982 627781
645992 655791
647572 657371
6485

In [11]:
smokes_new = pd.DataFrame(smokes_new)
smokes_new['thrower_name'] = events['player_name']
smokes_new = smokes_new.fillna(-2)
smokes_new.head(5)

,0,1,2,3,4,5,6,7,8,9,...,107794,107795,107796,107797,107798,107799,107800,107801,107802,thrower_name
0,-1031.319946,861.966980,11776.03125,100,0,200,200,5.0,12.0,0.0,...,4100,200,1.0,30.0,0.0,Feen,11778.031250,-301.967926,-887.135376,Fmgeez
1,-111.242317,-415.205200,11882.03125,100,0,200,200,5.0,12.0,0.0,...,4100,200,1.0,-1.0,0.0,Feen,11778.031250,578.336060,-1808.953003,P1STON
2,-833.233765,-173.878723,11776.03125,100,0,200,200,5.0,12.0,0.0,...,5100,4100,1.0,-1.0,0.0,Feen,11778.031250,506.728821,-2011.282959,Wargon
3,-732.479187,461.323944,11776.03125,100,0,4600,200,5.0,-1.0,0.0,...,5100,4100,1.0,30.0,0.0,Feen,11569.047852,-698.057373,-865.183594,Vladiclavchik
4,-955.005615,-696.865662,11848.06250,100,0,4600,200,5.0,20.0,0.0,...,5100,4100,1.0,30.0,0.0,Feen,-989.968750,-2653.332031,-577.540649,Fmgeez


In [12]:
categorical_columns = smokes_new.select_dtypes(include='object').columns
for column in categorical_columns:
    smokes_new[column] = smokes_new[column].map(country_map)

smokes_new.head(5)

,0,1,2,3,4,5,6,7,8,9,...,107794,107795,107796,107797,107798,107799,107800,107801,107802,thrower_name
0,-1031.319946,861.966980,11776.03125,100,0,200,200,5.0,12.0,0.0,...,4100,200,1.0,30.0,0.0,9,11778.031250,-301.967926,-887.135376,2
1,-111.242317,-415.205200,11882.03125,100,0,200,200,5.0,12.0,0.0,...,4100,200,1.0,-1.0,0.0,9,11778.031250,578.336060,-1808.953003,0
2,-833.233765,-173.878723,11776.03125,100,0,200,200,5.0,12.0,0.0,...,5100,4100,1.0,-1.0,0.0,9,11778.031250,506.728821,-2011.282959,8
3,-732.479187,461.323944,11776.03125,100,0,4600,200,5.0,-1.0,0.0,...,5100,4100,1.0,30.0,0.0,9,11569.047852,-698.057373,-865.183594,5
4,-955.005615,-696.865662,11848.06250,100,0,4600,200,5.0,20.0,0.0,...,5100,4100,1.0,30.0,0.0,9,-989.968750,-2653.332031,-577.540649,2


In [30]:

#smokes_new['stopped_rush'] = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
#smokes_new['label'] = [3,3,4,4,3,4,3,4,3,4,4,2,2,2,1,2,3,3,3,4,3,4,3,4]
#smokes_new['stopped_rush'] = [0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
#smokes_new['label'] = [3,3,3,4,4,3,5,3,3,4,3,3,4,2,3,3,3,2,3,2,3,4,4,4,2,2,3,3,3,2,3,3,2,3,3,4,3,3,4,3,3,3,3,4,3,3,3,4,2,3,3,4,3,3,4,4,3,3]
#smokes_new['stopped_rush'] = [0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
#smokes_new['label'] = [3,2,5,4,3,4,3,3,4,3,3,3,3,3,3,4,3,3,3,2,4,3,4,3,3,4,4,4,4,3,3,3,3,3,3,3,3,3,3,4,3,4,2,4,3,4,3,3,3,5,2,2,4,3,3,3,2,3,3,3,3,3,3,4,3,4,3,3,3,4,4,3,3,3,4,4,3,3,3,3,3,4,4,3]
smokes_new['stopped_rush'] = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
smokes_new['label'] = [3,3,3,3,3,3,4,4,3,4,4,3,4,3,3,4,2,3,5,3,3,3,3,3,3,4,3,3,3,4,4,3,4,3,4,4,3,3,3,4,3,3,3,4,3,2,3,3,3,3,3,4,2,3,3,4,3,3,3,4,4,3,4,3,4,2,3,4,3,3,2,2,3,3,3,3,2,2,2,3,3,3,4,3,4,2,3,3,3,4,3,4,3,3,2,3,4,4,3,3,3,4,3,2]
smokes_new.head(5)

C:\Users\tymot\AppData\Local\Temp\ipykernel_25332\2926328975.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  smokes_new['stopped_rush'] = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
C:\Users\tymot\AppData\Local\Temp\ipykernel_25332\2926328975.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  smokes_new['label'] = [3,3,3,3,3,3,4,4,3,4,4,3,4,3,3,4,2,3,5,3,3,3,3,

,0,1,2,3,4,5,6,7,8,9,...,107796,107797,107798,107799,107800,107801,107802,thrower_name,stopped_rush,label
0,-1772.563599,164.221420,11776.03125,100,0,850,200,-2.0,13.0,0.0,...,1.0,20.0,0.0,2,11746.03125,244.179291,-2386.478760,9,0,3
1,-592.483521,-242.981995,11776.03125,100,0,700,200,2.0,-1.0,0.0,...,3.0,-2.0,0.0,2,11605.84375,-144.030716,-2393.660400,6,0,3
2,-679.913208,-594.880066,11776.03125,100,0,700,200,2.0,7.0,0.0,...,3.0,-2.0,0.0,2,11778.03125,-492.871948,-250.106354,2,0,3
3,-1061.277588,-652.701172,11776.03125,100,0,700,200,2.0,7.0,0.0,...,3.0,-2.0,0.0,2,11778.03125,-341.490021,-436.412476,1,0,3
4,-1942.220093,106.523979,11776.03125,100,0,4500,200,3.0,9.0,0.0,...,3.0,-2.0,0.0,2,11778.03125,-637.747803,-1497.469849,3,0,3


In [151]:
#smokes.to_csv('smokes_data.csv',header='False', index='False', mode='a')

In [31]:
import csv

data = smokes_new.values.tolist()
with open('smokes_data.csv', 'a', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(data)

In [4]:
smokes = pd.read_csv('smokes_data.csv', header=None)
smokes = smokes.drop(0)
#smokes = smokes.drop('Unnamed: 0', axis=1)
smokes

,0,1,2,3,4,5,6,7,8,9,...,107796,107797,107798,107799,107800,107801,107802,107803,107804,107805
1,-2248.283691,561.440613,11772.245117,100.0,0.0,4300.0,200.0,1.0,-1.0,0.000000,...,4.0,-2.0,0.0,6.0,11778.031250,-1188.286011,-1032.276001,7.0,0.0,3.0
2,-2317.757812,456.750458,11744.031250,100.0,0.0,4300.0,200.0,1.0,20.0,2.659702,...,4.0,-2.0,0.0,6.0,11554.031250,-48.543045,-2385.574219,6.0,0.0,2.0
3,-1031.319946,861.966980,11776.031250,100.0,0.0,5100.0,4300.0,3.0,-1.0,0.000000,...,4.0,-2.0,0.0,6.0,11778.031250,-271.599365,-1500.390625,4.0,1.0,5.0
4,-2411.572510,300.580505,11744.031250,100.0,0.0,5100.0,4300.0,3.0,20.0,0.000000,...,4.0,-2.0,0.0,6.0,11778.031250,-162.787277,-1603.562866,0.0,0.0,4.0
5,-2247.439941,33.029839,11552.031250,100.0,0.0,5100.0,4300.0,3.0,20.0,0.000000,...,4.0,-2.0,0.0,6.0,11778.031250,-88.303665,-1956.900757,1.0,0.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
268,-1776.624512,-946.738464,11488.031250,100.0,0.0,5600.0,4600.0,26.0,-1.0,0.000000,...,18.0,-2.0,0.0,2.0,11778.031250,-1167.655518,-1041.045044,8.0,0.0,4.0
269,-957.977234,-1096.692749,11716.031250,100.0,0.0,5600.0,4600.0,26.0,30.0,0.000000,...,18.0,-2.0,0.0,2.0,11778.001953,-82.814743,-135.948776,0.0,0.0,3.0
270,-957.992493,-1112.093262,11769.871094,100.0,0.0,5600.0,4600.0,27.0,22.0,0.000000,...,18.0,-2.0,0.0,2.0,11778.031250,-115.919716,-341.304138,4.0,0.0,2.0
271,-1690.665527,946.746338,11808.171875,100.0,0.0,5700.0,2900.0,13.0,30.0,0.000000,...,7.0,-2.0,0.0,6.0,-989.968750,-2635.832764,-480.989960,0.0,0.0,2.0


In [5]:
from sklearn.model_selection import train_test_split
features = smokes.iloc[:, :-2]
labels = smokes[107805]


train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size=0.2, random_state=35)


In [77]:
pd.set_option('display.max_rows', 216)
display(test_features)
pd.reset_option('display.max_rows')

,0,1,2,3,4,5,6,7,8,9,...,107794,107795,107796,107797,107798,107799,107800,107801,107802,107803
77,-1806.626465,-1042.67688,11488.03125,100.0,0.0,3600.0,200.0,22.0,-1.0,0.0,...,5550.0,5050.0,14.0,12.0,0.0,6.0,11584.697266,-760.656189,-890.96582,5.0


In [16]:
survived = smokes[smokes[64014]==1]
deceased = smokes[smokes[64014]==0]
deceased = deceased.sample(n=len(survived), random_state=101)
dfy = pd.concat([survived,deceased],axis=0)

In [36]:
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_absolute_error

label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

model = xgb.XGBClassifier()
model.fit(train_features, train_labels_encoded) #xgb_model=r"C:\Users\tymot\Desktop\Projekty\Prawdopodobieństwa fragów\xgboostModel")

pred_labels_encoded = model.predict(test_features)
pred_labels = label_encoder.inverse_transform(pred_labels_encoded)

accuracy = accuracy_score(test_labels, pred_labels)
print('accuracy:',accuracy)
accuracy = mean_absolute_error(test_labels, pred_labels)
print('mse:',accuracy)


accuracy: 1.0
mse: 0.0


In [6]:
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_absolute_error

#Zamień 1,2,3,4,5 na 0,1,2,3,4 
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='softmax', input_shape=(107804,)),
    tf.keras.layers.Dense(64, activation='softmax'),
    tf.keras.layers.Dense(5, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(train_features, train_labels_encoded, epochs=100)

pred_labels_encoded = model.predict(test_features)
pred_labels_encoded = tf.argmax(pred_labels_encoded, axis=1)
pred_labels = label_encoder.inverse_transform(pred_labels_encoded)

accuracy = accuracy_score(test_labels, pred_labels)
print('accuracy:', accuracy)

mae = mean_absolute_error(test_labels, pred_labels)
print('MAE:', mae)


Epoch 1/10
7/7 [==============================] - 3s 7ms/step - loss: 1.5930 - accuracy: 0.5853
Epoch 2/10
7/7 [==============================] - 0s 8ms/step - loss: 1.5808 - accuracy: 0.5853
Epoch 3/10
7/7 [==============================] - 0s 7ms/step - loss: 1.5684 - accuracy: 0.5853
Epoch 4/10
7/7 [==============================] - 0s 8ms/step - loss: 1.5567 - accuracy: 0.5853
Epoch 5/10
7/7 [==============================] - 0s 8ms/step - loss: 1.5446 - accuracy: 0.5853
Epoch 6/10
7/7 [==============================] - 0s 7ms/step - loss: 1.5334 - accuracy: 0.5853
Epoch 7/10
7/7 [==============================] - 0s 6ms/step - loss: 1.5219 - accuracy: 0.5853
Epoch 8/10
7/7 [==============================] - 0s 6ms/step - loss: 1.5108 - accuracy: 0.5853
Epoch 9/10
7/7 [==============================] - 0s 6ms/step - loss: 1.4999 - accuracy: 0.5853
Epoch 10/10
2/2 [==============================] - 0s 6ms/step
accuracy: 0.6181818181818182
MAE: 0.38181818181818183


In [25]:
new_data = smokes_new.iloc[35]
new_data = pd.DataFrame(new_data.values)
new_data = new_data.T

model.predict(new_data)


1/1 [==============================] - 0s 17ms/step


array([[0.17745386, 0.18216816, 0.24254365, 0.22196367, 0.17587069]],
      dtype=float32)

In [39]:
smokes_new

,0,1,2,3,4,5,6,7,8,9,...,107796,107797,107798,107799,107800,107801,107802,thrower_name,stopped_rush,label
0,-1772.563599,164.221420,11776.031250,100,0,850,200,-2.0,13.0,0.0,...,1.0,20.0,0.0,2,11746.031250,244.179291,-2386.478760,9,0,3
1,-592.483521,-242.981995,11776.031250,100,0,700,200,2.0,-1.0,0.0,...,3.0,-2.0,0.0,2,11605.843750,-144.030716,-2393.660400,6,0,3
2,-679.913208,-594.880066,11776.031250,100,0,700,200,2.0,7.0,0.0,...,3.0,-2.0,0.0,2,11778.031250,-492.871948,-250.106354,2,0,3
3,-1061.277588,-652.701172,11776.031250,100,0,700,200,2.0,7.0,0.0,...,3.0,-2.0,0.0,2,11778.031250,-341.490021,-436.412476,1,0,3
4,-1942.220093,106.523979,11776.031250,100,0,4500,200,3.0,9.0,0.0,...,3.0,-2.0,0.0,2,11778.031250,-637.747803,-1497.469849,3,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,-1039.822998,95.225456,11776.031250,97,0,5000,200,26.0,22.0,0.0,...,18.0,12.0,0.0,2,11778.031250,597.028870,-1792.298096,4,0,3
100,-1770.726807,-1035.267334,11488.031250,100,0,5600,4600,26.0,-1.0,0.0,...,18.0,-2.0,0.0,2,11537.650391,-572.469604,-758.566711,6,0,3
101,-1776.624512,-946.738464,11488.031250,100,0,5600,4600,26.0,-1.0,0.0,...,18.0,-2.0,0.0,2,11778.031250,-1167.655518,-1041.045044,8,0,4
102,-957.977234,-1096.692749,11716.031250,100,0,5600,4600,26.0,30.0,0.0,...,18.0,-2.0,0.0,2,11778.001953,-82.814743,-135.948776,0,0,3


In [151]:
smokes_new['map'] = smokes_new['map'].astype('category').cat.codes

In [41]:
new_data

,0,1,2,3,4,5,6,7,8,9,...,107796,107797,107798,107799,107800,107801,107802,107803,107804,107805
0,-2355.774414,-44.634747,11552.03125,0.0,0.0,5200.0,4700.0,25.0,-2.0,0.0,...,18.0,-2.0,0.0,2.0,11778.03125,577.594788,-1794.603149,4.0,0.0,4.0


In [106]:
new_data = smokes_new.iloc[34]
new_data = pd.DataFrame(new_data.values)
new_data = new_data.T
new_data[107804] = 0
new_data[107805] = 4
new_data
data = new_data.values.tolist()

with open('smokes_data.csv', 'a', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(data)

In [86]:
new_data

,0,1,2,3,4,5,6,7,8,9,...,107794,107795,107796,107797,107798,107799,107800,107801,107802,107803
0,-1690.665527,946.746338,11808.171875,100.0,0.0,5700.0,2900.0,13.0,30.0,0.0,...,500.0,200.0,7.0,-2.0,0.0,6.0,-989.96875,-2635.832764,-480.98996,0.0


In [40]:
#new_data = smokes.iloc[155].drop([107804,107805])
new_data = smokes_new.iloc[31]
new_data = pd.DataFrame(new_data.values)
new_data = new_data.T
temp = model.predict_proba(new_data)
temp = temp[0]

suma = 0
iter = 1
for i in temp:
    suma += i * iter
    iter += 1
print(suma, temp)

3.2760836891829967 [0.01032191 0.05384585 0.5957232  0.3296446  0.01046441]


In [49]:
#new_data = smokes.iloc[84].drop([107805,107806])
for i in range(len(smokes_new)):
    new_data = smokes_new.iloc[i]
    new_data = pd.DataFrame(new_data.values)
    new_data = new_data.T
    temp = model.predict_proba(new_data)
    temp = temp[0]

    suma = 0
    iter = 1
    for j in temp:
        suma += j * iter
        iter += 1
    print(i, suma, temp)


0 3.03533860493917 [0.00107854 0.00337131 0.95586485 0.03850321 0.00118202]
1 3.084910752950236 [0.00329724 0.0270758  0.8565954  0.10748198 0.00554956]
2 3.010750892572105 [0.01189802 0.08312225 0.7944909  0.10330837 0.00718045]
3 3.0404192893765867 [0.00183817 0.00416966 0.9485116  0.04269617 0.00278447]
4 3.595409980043769 [0.00535868 0.02276712 0.35135496 0.6121441  0.00837515]
5 3.040850313846022 [0.00463426 0.0202319  0.9101862  0.05954452 0.00540313]
6 3.0281200321624056 [0.00208083 0.00278477 0.96157753 0.03204762 0.00150932]
7 3.012029941135552 [9.8945771e-04 1.6164141e-03 9.8255712e-01 1.4048947e-02 7.8810268e-04]
8 3.032109455554746 [0.00183764 0.01720752 0.9298207  0.04927613 0.00185802]
9 3.10339609766379 [0.00200019 0.00374154 0.8860567  0.10526537 0.00293624]
10 3.027095902711153 [0.0017851  0.00191236 0.9653324  0.02936164 0.00160847]
11 3.024694452062249 [0.00161162 0.00442275 0.9647858  0.02601912 0.00316068]
12 3.0040614067111164 [0.00127508 0.00378928 0.98703533 0.0

In [9]:
new_data

,0,1,2,3,4,5,6,7,8,9,...,64004,64005,64006,64007,64008,64009,64010,64011,64012,64013
0,-971.916504,757.763428,11776.03125,100.0,0.0,5800.0,4400.0,6.0,-1.0,0.0,...,0.0,5800.0,4400.0,6.0,-2.0,0.0,11531.902344,-549.474792,-772.823547,0.0


In [63]:
smokes.value_counts([107805])

107805
3.0       161
4.0        74
2.0        30
5.0         4
1.0         1
dtype: int64

In [13]:
model.save_model("xgboostModel")

In [ ]:
pd.set_option('display.max_rows', 277)
display(smokes)
pd.reset_option('display.max_rows')